Сухоруков Кирилл Евгеньевич 

# Импорт библиотек и определение функций генерации матриц



In [1]:
import numpy as np
import time
import torch

In [2]:
def gen_matr_CPU(n):
  A=np.random.random([n,n])
  b=np.random.random([n])
  return A, b

In [3]:
def gen_matr_GPU(n):
  torch.cuda.get_device_name(0)
  A=torch.rand([n,n], dtype=torch.float32).cuda()
  b=torch.rand([n,1], dtype=torch.float32).cuda()
  return A, b

In [4]:
def transf_to_torch(A, b):
  A_torch=torch.from_numpy(A).cuda()
  b_torch=torch.unsqueeze(torch.from_numpy(b), 1).cuda()
  return A_torch, b_torch

In [5]:
solution_time_CPU=[]
solution_time_GPU=[]

# Решение на NumPy

In [6]:
n=2000
A_CPU1, b_CPU1=gen_matr_CPU(n)

In [7]:
t0=time.time()
x_CPU=np.linalg.solve(A_CPU1, b_CPU1)
t1=time.time()
print(f'RunTime: {t1-t0}')
solution_time_CPU.append(t1-t0)

RunTime: 0.23944664001464844


In [8]:
n=5000
A_CPU2, b_CPU2=gen_matr_CPU(n)

In [9]:
t0=time.time()
x_CPU=np.linalg.solve(A_CPU2, b_CPU2)
t1=time.time()
print(f'RunTime: {t1-t0}')
solution_time_CPU.append(t1-t0)

RunTime: 2.9390697479248047


In [10]:
n=10000
A_CPU3, b_CPU3=gen_matr_CPU(n)

In [11]:
t0=time.time()
x_CPU=np.linalg.solve(A_CPU3, b_CPU3)
t1=time.time()
print(f'RunTime: {t1-t0}')
solution_time_CPU.append(t1-t0)
solution_time_CPU=np.array(solution_time_CPU)

RunTime: 20.892002820968628


# Решение на PyTorch

In [12]:
torch.cuda.get_device_name(0)

'Tesla K80'

In [13]:
A_GPU1, b_GPU1=transf_to_torch(A_CPU1, b_CPU1)

In [14]:
t0=time.time()
x_GPU=torch.solve(b_GPU1, A_GPU1).solution
t1=time.time()
print(f'RunTime: {t1-t0}')
solution_time_GPU.append(t1-t0)

RunTime: 0.2641637325286865


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: UserWarning: torch.solve is deprecated in favor of torch.linalg.solveand will be removed in a future PyTorch release.
torch.linalg.solve has its arguments reversed and does not return the LU factorization.
To get the LU factorization see torch.lu, which can be used with torch.lu_solve or torch.lu_unpack.
X = torch.solve(B, A).solution
should be replaced with
X = torch.linalg.solve(A, B) (Triggered internally at  /pytorch/aten/src/ATen/native/BatchLinearAlgebra.cpp:760.)
  


In [15]:
A_GPU2, b_GPU2=transf_to_torch(A_CPU2, b_CPU2)

In [16]:
t0=time.time()
x_GPU=torch.solve(b_GPU2, A_GPU2).solution
t1=time.time()
print(f'RunTime: {t1-t0}')
solution_time_GPU.append(t1-t0)

RunTime: 0.2466719150543213


In [17]:
A_GPU3, b_GPU3=transf_to_torch(A_CPU3, b_CPU3)

In [18]:
t0=time.time()
x_GPU=torch.solve(b_GPU3, A_GPU3).solution
t1=time.time()
print(f'RunTime: {t1-t0}')
solution_time_GPU.append(t1-t0)
solution_time_GPU=np.array(solution_time_GPU)

RunTime: 1.3769991397857666


In [19]:
n_list=['n = 2000', 'n = 5000', 'n = 10000']
solution_time_dif=solution_time_CPU-solution_time_GPU
max_diff=solution_time_dif.argmax()
print(f'Наибольший прирост производительности произошел при {n_list[max_diff]} и равен {solution_time_dif.max()} сек.')

Наибольший прирост производительности произошел при n = 10000 и равен 19.51500368118286 сек.


# Поиск минимального n при котором GPU начинает превосходить CPU


In [30]:
n=500
flag=0
while (True):
    A_CPU, b_CPU=gen_matr_CPU(n)
    t0_CPU=time.time()
    x_CPU=np.linalg.solve(A_CPU, b_CPU)
    t1_CPU=time.time()
    A_GPU, b_GPU=transf_to_torch(A_CPU, b_CPU)
    t0_GPU=time.time()
    x_GPU=torch.solve(b_GPU, A_GPU).solution
    t1_GPU=time.time()

    t_CPU=t1_CPU-t0_CPU
    t_GPU=t1_GPU-t0_GPU
    
    
    if (t_CPU-t_GPU>0.0 and not flag):
      print(f'GPU начинает превосходить CPU в производительности начиная с ~{n} элементов')
      flag=1
    if (t_CPU-t_GPU>1.0):
      break
    n+=100
print(f'GPU начинает превосходить CPU в производительности более чем на 1 секунду начиная с ~{n} элементов')

GPU начинает превосходить CPU в производительности начиная с ~800 элементов
GPU начинает превосходить CPU в производительности более чем на 1 секунду начиная с ~3700 элементов
